In [16]:
import numpy as np
import json as js
import matplotlib.pyplot as plt
import os
import tensorflow as tf

In [3]:
#load data

base_dir = "C:\\Users\\1\\Desktop\\datasets"
data_buffer = []
max_data = 2000

for json_file in os.listdir(base_dir)[:-1]:


    curent_file = os.path.join(base_dir, json_file)
    curent_data_buffer = {}
    with open (curent_file) as file:

        json_data = file.readlines()

        for (json_number, json_per_line) in enumerate(json_data):
            
            if json_number == max_data:
                
                break

            json_format = js.loads(json_per_line)
            curent_data_buffer[f"subject_number: {json_number}"] = json_format
            max_data += 1
    
    data_buffer.append(curent_data_buffer)


In [4]:
samples_data_tensor = []
for (batch_number, data_batch) in enumerate(data_buffer):

    for (subject_number, subject) in enumerate(data_batch):

        try:

            for sample in data_batch[subject]["steps"]["samples"]:


                sample_vector = np.asarray([float(feature) for feature in sample.values()])
                sample_vector[0] *= (10 ** -5)
                sample_vector[1] /= 120.0

                if batch_number == 0:

                    add_vector = np.asarray([
                        float(data_batch[subject]["steps"]["steps"]),
                        float(data_batch[subject]["steps"]["day"]),
                        float(data_batch[subject]["steps"]["meters"]),
                        0
                    ])
                    
                else:

                    add_vector = np.asarray([
                        float(data_batch[subject]["steps"]["steps"]),
                        float(data_batch[subject]["steps"]["day"]),
                        float(data_batch[subject]["steps"]["meters"]),
                        1
                    ])
                    
                result_features_vector = np.concatenate((sample_vector, add_vector))
                samples_data_tensor.append(result_features_vector)
        
        except BaseException:

            pass
            


samples_data_tensor = np.asarray(samples_data_tensor)
print(samples_data_tensor)





[[1.6906149e+07 7.5000000e+03 1.0000000e+01 ... 1.3050000e+03
  1.0190000e+04 0.0000000e+00]
 [1.6906158e+07 7.5000000e+03 1.3000000e+01 ... 1.3050000e+03
  1.0190000e+04 0.0000000e+00]
 [1.6906167e+07 7.5000000e+03 7.0000000e+00 ... 1.3050000e+03
  1.0190000e+04 0.0000000e+00]
 ...
 [1.7056539e+07 7.5000000e+03 1.8400000e+02 ... 1.9741000e+04
  9.4800000e+02 1.0000000e+00]
 [1.7056557e+07 7.5000000e+03 1.8400000e+02 ... 1.9741000e+04
  9.4800000e+02 1.0000000e+00]
 [1.7056764e+07 7.5000000e+03 1.4700000e+02 ... 1.9741000e+04
  9.4800000e+02 1.0000000e+00]]


In [15]:
permutated_data_tensor = np.random.permutation(samples_data_tensor)

train_data = permutated_data_tensor[:samples_data_tensor.shape[0] // 2, :-1]
train_labels = permutated_data_tensor[:samples_data_tensor.shape[0] // 2, -1]

validation_data = permutated_data_tensor[samples_data_tensor.shape[0] // 2:, :-1]
validation_labels = permutated_data_tensor[samples_data_tensor.shape[0] // 2:, -1]

print(f"Train tensor shape: {train_data.shape}, \n Test tensor shape: {validation_data.shape}")
print(f" \nData example: {permutated_data_tensor} \n Labels example: \t{validation_labels.shape}")


Train tensor shape: (17656911, 6), 
 Test tensor shape: (17656912, 6)
 
Data example: [[1.6963218e+07 7.5000000e+03 2.2200000e+02 ... 1.9633000e+04
  1.1570000e+04 1.0000000e+00]
 [1.6879635e+07 7.5000000e+03 1.6370000e+03 ... 1.2740000e+03
  5.8660000e+03 1.0000000e+00]
 [1.6996437e+07 7.5000000e+03 2.6000000e+01 ... 1.9671000e+04
  2.0680000e+03 1.0000000e+00]
 ...
 [1.6936461e+07 7.5000000e+03 2.4600000e+02 ... 1.9602000e+04
  1.7397000e+04 1.0000000e+00]
 [1.6952220e+07 7.5000000e+03 3.3000000e+02 ... 1.9620000e+04
  5.9390000e+03 1.0000000e+00]
 [1.6883937e+07 7.5000000e+03 1.9000000e+01 ... 1.2790000e+03
  1.5169000e+04 1.0000000e+00]] 
 Labels example: 	(17656912,)
0 label was found!!!


In [19]:
input_tensor = tf.keras.Input(shape=(6, ))

layer = tf.keras.layers.Dense(100, activation="relu")(input_tensor)
layer = tf.keras.layers.Dense(64, activation="relu")(layer)
layer = tf.keras.layers.Dense(64, activation="relu")(layer)
layer = tf.keras.layers.Dense(32, activation="relu")(layer)
layer = tf.keras.layers.Dense(32, activation="relu")(layer)
layer = tf.keras.layers.Dense(10, activation="relu")(layer)
last_layer = tf.keras.layers.Dense(1, activation="sigmoid")(layer)

model = tf.keras.Model(input_tensor, last_layer)
model.compile(
    optimizer=tf.optimizers.RMSprop(learning_rate=0.01),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=tf.keras.metrics.Accuracy()
)


In [ ]:
model_history = model.fit(train_data, train_labels,
                          epochs=100,
                          batch_size=30,
                          validation_data=(validation_data, validation_labels))
model.save("C:\\Users\\1\\Desktop\\ItPLanetProject2\\SavedModels\\first_model.keras")
